In [1]:
import pandas as pd
import os
import numpy as np

# Parse through precipitation data & extract only dates in labeled range

In [2]:
directory = r"C:\Users\olive\Documents\GitHub\MilaClimateChangeHackathon2019\data"
df = pd.DataFrame()
for dirpath, dirnames, filenames in os.walk(directory):
    for file in filenames:
        if '_standard.csv' in file:
            abspath = dirpath + '\\' + file
            dfT = pd.read_csv(abspath, index_col='time', parse_dates=True)
            dfT = dfT.loc['2012':'2017']
            df = pd.concat([df, dfT])

C:\Users\olive\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


## One-Hot Encode Grid for Features

In [3]:
# Add column Grid to keep track of what grid the data is from
df['Grid'] = df['lat'].astype('str') + '|' + df['lon'].astype('str')
# One-Hot Encode Grid to use as feature later
df = pd.concat([df, pd.get_dummies(df['Grid'], prefix='GRID', drop_first=True)], axis=1)

## Define Method to Snap Lat/Long to Grid

In [4]:
def closest_grid(point, data):
    min_dist = 1_000_000
    for grid in data:
        dist = np.sum((point - grid)**2)
        if dist < min_dist:
            min_dist = dist
            best_grid = grid
    return best_grid[0], best_grid[1]

In [5]:
grid_data = []
for grid in np.unique(df['Grid']):
    lat, lon = grid.split('|')
    lat, lon = float(lat), float(lon)
    grid_data.append(np.array([lat, lon]))
point = np.array([45.5, -73.9])
closest_grid(point, grid_data)

(45.541666485, -73.875002685)

## One-Hot Encode Weekday & Month for Features

In [6]:
df['Weekday'] = df.index.weekday
df = pd.concat([df.drop(['Weekday'], axis=1), pd.get_dummies(df['Weekday'], prefix='WEEKDAY', drop_first=True)], axis=1)

df['Month'] = df.index.month
df = pd.concat([df.drop(['Month'], axis=1), pd.get_dummies(df['Month'], prefix='MONTH', drop_first=True)], axis=1)

## Filter unused grids

In [7]:
acc_df = pd.read_csv(r"C:\Users\olive\Documents\GitHub\MilaClimateChangeHackathon2019\data\Accident_ResponseData_WithGrids.csv")
used_grids = np.unique(acc_df['Grid'])
used_grids = [x.replace('GRID_', '') for x in used_grids]

In [8]:
df = df.loc[df['Grid'].isin(used_grids)]

## Set Index on Time/Lat/Lon

In [9]:
df = df.reset_index()
df = df.sort_values(['time', 'Grid']).set_index(['time','Grid'])

## Dump to CSV

In [10]:
df.to_csv(r"C:\Users\olive\Documents\GitHub\MilaClimateChangeHackathon2019\data\prepared_precipitation_data.csv")